In [39]:
import time

from domain_detection import Detector
from retrieval import Retriever
from llm import LLM, BaseLLM

import warnings
warnings.filterwarnings('ignore')

In [38]:
LLM_INPUT_TEMPLATE = '''
<s><|im_start|>system
{system_prompt}
</s><|im_start|>user
{user_query}
</s><|im_start|>assistant
'''

DEFAULT_SYSTEM_PROMPT_TEMPLATE = '''
คุณคือนักกฎหมายที่จะตอบคำถามเกี่ยวกับกฎหมาย จงตอบคำถามโดยใช้ความรู้ที่ให้ดังต่อไปนี้
ถ้าหากคุณไม่รู้คำตอบ ให้ตอบว่าไม่รู้ อย่าสร้างคำตอบขึ้นมาเอง
ความรู้ที่ให้:
{context}
'''

DEFAULT_BASE_SYSTEM_PROMPT_TEMPLATE = '''
คุณคือนักกฎหมายที่จะตอบคำถามทั่วไปด้วยความสุภาพ
'''

WARMUP_INPUT = 'การขับรถชนคนโดยไม่ตั้งใจผิดกฎหมายไหม'


class LegalAssistant:

    def __init__(
        self,
        # Domain Detection part
        domain_detection_model_path='/project/lt200301-edubot/Capstone-TamTanai/models/finetuned_mpnet',
        # Retrieval part
        specific_case_path='/home/shanwibo/Capstone-TamTanai/notebooks/specific_case_knowledge.txt',
        general_source_dir='/home/shanwibo/Capstone-TamTanai/asset/documentation',
        keyword_search=True,
        idf=True,
        case_number=True,
        law_number=True,
        law_name=True,
        context_search=True,
        embedding_model_path='/project/lt200301-edubot/Capstone-TamTanai/models/multilingual-e5-large',
        persist_directory=None,
        vector_store='faiss',
        similarity_threshold=0.6,
        n_similar_documents=40,
        reranker_path='/project/lt200301-edubot/Capstone-TamTanai/models/bge-reranker-v2-m3-finetune-with_similar=5_keyword=5_2nd',
        # LLM part
        system_prompt_template=DEFAULT_SYSTEM_PROMPT_TEMPLATE,
        finetuned_llm_name='typhoon2-qwen2.5-7b-instruct-finetune-dataset-v1-exam-thailegal-with-v1+exam+thailegal-validation-set',
        base_system_prompt_template=DEFAULT_BASE_SYSTEM_PROMPT_TEMPLATE,
        base_llm_name='typhoon2-qwen2.5-7b-instruct'
    ):
        self.domain_detector = Detector(model_path=domain_detection_model_path)
        retrieval_kwargs = {
            'specific_case_path': specific_case_path,
            'general_source_dir': general_source_dir,
            'keyword_search': keyword_search,
            'idf': idf,
            'case_number': case_number,
            'law_number': law_number,
            'law_name': law_name,
            'context_search': context_search,
            'embedding_model_path': embedding_model_path,
            'persist_directory': persist_directory,
            'vector_store': vector_store,
            'similarity_threshold': similarity_threshold,
            'n_similar_documents': n_similar_documents,
            'reranker_path': reranker_path,
        }
        self.retriever = Retriever(**retrieval_kwargs)
        self.llm = LLM(model_name=finetuned_llm_name)
        self.base_llm = BaseLLM(model_name=base_llm_name)
        self.system_prompt_template = system_prompt_template
        self.base_system_prompt_template = base_system_prompt_template
        self._warmup()

    def _warmup(self):
        self.domain_detector(WARMUP_INPUT)
        self.retriever.search(WARMUP_INPUT)

    def __call__(self, text, n_documents=1):
        processing_start = time.time()
        domain_detection_start = time.time()
        legal_domain, legal_domain_prob = self.domain_detector(text, return_prob=True)
        domain_detection_end = time.time()
        domain_detection = {
            'result': legal_domain,
            'probability': legal_domain_prob,
            'time_usage': domain_detection_end - domain_detection_start
        }
        if legal_domain:
            retrieval_start = time.time()
            retrieval_results = self.retriever.search(text, n=n_documents)
            retrieval_end = time.time()
            retrieval = {
                'result': [retrieval_result['document'].page_content for retrieval_result in retrieval_results],
                'reranking_score': [retrieval_result['reranking_score'] for retrieval_result in retrieval_results],
                'time_usage': retrieval_end - retrieval_start
            }
            context = '\n\n\n'.join([retrieval_result['document'].page_content for retrieval_result in retrieval_results])
            system_prompt = self.system_prompt_template.format(context=context)
            llm_input = LLM_INPUT_TEMPLATE.format(system_prompt=system_prompt, user_query=text)
            generation_start = time.time()
            response = self.llm(llm_input)
            if '</s>' in response:
                response = response[: response.index('</s>')]
            generation_end = time.time()
            generation = {
                'result': response,
                'time_usage': generation_end - generation_start
            }
        else:
            retrieval = None
            llm_input = LLM_INPUT_TEMPLATE.format(system_prompt=self.base_system_prompt_template, user_query=text)
            generation_start = time.time()
            response = self.base_llm(llm_input)
            generation_end = time.time()
            generation = {
                'result': response,
                'time_usage': generation_end - generation_start
            }
        processing_end = time.time()
        
        return {
            'domain_detection': domain_detection,
            'retrieval': retrieval,
            'generation': generation,
            'processing_time': processing_end - processing_start
        }

In [18]:
legal_assistant = LegalAssistant()

----------using 4*GPUs----------


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.22s/it]


In [19]:
text = 'ขับรถชนคนผิดไหม'

In [24]:
result = legal_assistant('สิทธิในการลาคลอดบุตรของพนักงานหญิง', n_documents=1)
result

{'domain_detection': {'result': 1,
  'probability': 0.9997760653495789,
  'time_usage': 0.008022785186767578},
 'retrieval': {'result': ['พระราชบัญญัติคุ้มครองแรงงาน (ฉบับที่ 8) พ.ศ. 2566 - หมวด 3 (การใช้แรงงานหญิง)\n\nมาตรา 38   ห้ามมิให้นายจ้างให้ลูกจ้างซึ่งเป็นหญิงทำงานอย่างหนึ่งอย่างใด ดังต่อไปนี้\n(1) งานเหมืองแร่หรืองานก่อสร้างที่ต้องทำใต้ดิน ใต้น้ำ ในถ้ำ ในอุโมงค์หรือปล่องในภูเขา เว้นแต่สภาพของการทำงานไม่เป็นอันตรายต่อสุขภาพหรือร่างกายของลูกจ้าง\n(2) งานที่ต้องทำบนนั่งร้านที่สูงกว่าพื้นดินตั้งแต่สิบเมตรขึ้นไป\n(3) งานผลิตหรือขนส่งวัตถุระเบิดหรือวัตถุไวไฟ เว้นแต่สภาพของการทำงานไม่เป็นอันตรายต่อสุขภาพหรือร่างกายของลูกจ้าง\n(4) งานอื่นตามที่กำหนดในกฎกระทรวง\n\nมาตรา 39   ห้ามมิให้นายจ้างให้ลูกจ้างซึ่งเป็นหญิงมีครรภ์ทำงานอย่างหนึ่งอย่างใด ดังต่อไปนี้\n(1) งานเกี่ยวกับเครื่องจักรหรือเครื่องยนต์ที่มีความสั่นสะเทือน\n(2) งานขับเคลื่อนหรือติดไปกับยานพาหนะ\n(3) งานยก แบก หาม หาบ ทูน ลาก หรือเข็นของหนักเกินสิบห้ากิโลกรัม\n(4) งานที่ทำในเรือ\n(5) งานอื่นตามที่กำหนดในกฎกระทรวง\n\nมาตรา 39/1

In [11]:
result = legal_assistant(text, n_documents=2)
result

{'domain_detection': {'result': 1,
  'probability': 0.9998040795326233,
  'time_usage': 0.008005142211914062},
 'retrieval': {'result': ['พระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565 - ลักษณะ 3 (การใช้ทางเดินรถ) - หมวด 1 (การขับรถ)\n\nมาตรา 31  นอกจากที่บัญญัติไว้เป็นพิเศษในลักษณะ 4 ว่าด้วยการใช้ทางเดินรถที่จัดเป็นช่องเดินรถประจำทาง การใช้ทางเดินรถให้เป็นไปตามที่บัญญัติไว้ในลักษณะนี้\n\nมาตรา 31/1  ในขณะขับรถในทางเดินรถ ผู้ขับขี่ต้องมีใบอนุญาตขับขี่อยู่กับตัวและต้องแสดงต่อเจ้าพนักงานจราจรเมื่อขอตรวจ\nในกรณีที่ผู้ขับขี่แสดงใบอนุญาตขับขี่ด้วยวิธีการทางข้อมูลอิเล็กทรอนิกส์หรือสำเนาภาพถ่ายใบอนุญาตขับขี่ตามที่กรมการขนส่งทางบกกำหนด ให้ถือว่าผู้ขับขี่มีใบอนุญาตขับขี่อยู่กับตัวตามวรรคหนึ่งแล้ว\n\nมาตรา 32  ในการใช้ทางเดินรถผู้ขับขี่ต้องใช้ความระมัดระวังไม่ให้รถชนหรือโดนคนเดินเท้าไม่ว่าจะอยู่ ณ ส่วนใดของทาง และต้องให้สัญญาณเตือนคนเดินเท้าให้รู้ตัวเมื่อจำเป็น โดยเฉพาะอย่างยิ่ง เด็ก คนชราหรือคนพิการที่กำลังใช้ทาง ผู้ขับขี่ต้องใช้ความระมัดระวังเป็นพิเศษในการควบคุมรถของตน\n\nมาตรา 33  ในการขับรถ 

In [28]:
LLM_INPUT_TEMPLATE = '''
<s><|im_start|>system
{system_prompt}
</s><|im_start|>user
{user_query}
</s><|im_start|>assistant
'''

In [34]:
text

'ขับรถชนคนผิดไหม'

In [29]:
legal_domain = legal_assistant.domain_detector(text)
if legal_domain:
    retrieval_results = legal_assistant.retriever.search(text, n=2)
    system_prompt = DEFAULT_SYSTEM_PROMPT_TEMPLATE.format(context=retrieval_results[0]['document'].page_content)
    llm_input = LLM_INPUT_TEMPLATE.format(system_prompt=system_prompt, user_query=text)

In [35]:
[retrieval_result['document'].page_content for retrieval_result in retrieval_results][1]

'พระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565 - ลักษณะ 9 (อุบัติเหตุ)\n\nมาตรา 78  ผู้ใดขับรถหรือขี่หรือควบคุมสัตว์ในทางซึ่งก่อให้เกิดความเสียหายแก่บุคคลหรือทรัพย์สินของผู้อื่นไม่ว่าจะเป็นความผิดของผู้ขับขี่หรือผู้ขี่หรือควบคุมสัตว์หรือไม่ก็ตาม ต้องหยุดรถ หรือสัตว์ และให้ความช่วยเหลือตามสมควร และพร้อมทั้งแสดงตัวและแจ้งเหตุต่อตำรวจ*ที่ใกล้เคียงทันที กับต้องแจ้งชื่อตัว ชื่อสกุล และที่อยู่ของตนและหมายเลขทะเบียนรถแก่ผู้ได้รับความเสียหายด้วย\nในกรณีที่ผู้ขับขี่หรือผู้ขี่หรือควบคุมสัตว์หลบหนีไปหรือไม่แสดงตัวต่อตำรวจ* ณ สถานที่เกิดเหตุ ให้สันนิษฐานว่าเป็นผู้กระทำความผิดและให้ตำรวจ*มีอำนาจยึดรถคันที่ผู้ขับขี่หลบหนีหรือไม่แสดงตนว่าเป็นผู้ขับขี่ จนกว่าคดีถึงที่สุดหรือได้ตัวผู้ขับขี่ ถ้าเจ้าของหรือผู้ครอบครองไม่แสดงตัวต่อตำรวจ*ภายในหกเดือนนับแต่วันเกิดเหตุ ให้ถือว่ารถนั้นเป็นทรัพย์สินซึ่งได้ใช้ในการกระทำความผิดหรือเกี่ยวกับการกระทำความผิด และให้ตกเป็นของรัฐ'

In [36]:
[retrieval_result['reranking_score'] for retrieval_result in retrieval_results]

[0.97369471946558, 0.0011203773223969365]

In [40]:
context = '\n\n\n'.join([retrieval_result['document'].page_content for retrieval_result in retrieval_results])

In [41]:
print(context)

พระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565 - ลักษณะ 3 (การใช้ทางเดินรถ) - หมวด 1 (การขับรถ)

มาตรา 31  นอกจากที่บัญญัติไว้เป็นพิเศษในลักษณะ 4 ว่าด้วยการใช้ทางเดินรถที่จัดเป็นช่องเดินรถประจำทาง การใช้ทางเดินรถให้เป็นไปตามที่บัญญัติไว้ในลักษณะนี้

มาตรา 31/1  ในขณะขับรถในทางเดินรถ ผู้ขับขี่ต้องมีใบอนุญาตขับขี่อยู่กับตัวและต้องแสดงต่อเจ้าพนักงานจราจรเมื่อขอตรวจ
ในกรณีที่ผู้ขับขี่แสดงใบอนุญาตขับขี่ด้วยวิธีการทางข้อมูลอิเล็กทรอนิกส์หรือสำเนาภาพถ่ายใบอนุญาตขับขี่ตามที่กรมการขนส่งทางบกกำหนด ให้ถือว่าผู้ขับขี่มีใบอนุญาตขับขี่อยู่กับตัวตามวรรคหนึ่งแล้ว

มาตรา 32  ในการใช้ทางเดินรถผู้ขับขี่ต้องใช้ความระมัดระวังไม่ให้รถชนหรือโดนคนเดินเท้าไม่ว่าจะอยู่ ณ ส่วนใดของทาง และต้องให้สัญญาณเตือนคนเดินเท้าให้รู้ตัวเมื่อจำเป็น โดยเฉพาะอย่างยิ่ง เด็ก คนชราหรือคนพิการที่กำลังใช้ทาง ผู้ขับขี่ต้องใช้ความระมัดระวังเป็นพิเศษในการควบคุมรถของตน

มาตรา 33  ในการขับรถ ผู้ขับขี่ต้องขับรถในทางเดินรถด้านซ้ายและต้องไม่ล้ำกึ่งกลางของทางเดินรถ เว้นแต่ในกรณีต่อไปนี้ ให้เดินทางขวาหรือล้ำกึ่งกลางของทางเดินรถได้
(1) ด้านซ้

In [11]:
if '</s>' in response:
    print(response[: response.index('</s>')])

ขับรถชนคนผิดไหม คำอธิบาย: มาตรา 31 กำหนดให้ผู้ขับขี่ต้องใช้ความระมัดระวังไม่ให้รถชนหรือโดนคนเดินเท้าไม่ว่าจะอยู่ ณ ส่วนใดของทาง และต้องให้สัญญาณเตือนคนเดินเท้าให้รู้ตัวเมื่อจำเป็น โดยเฉพาะอย่างยิ่ง เด็ก คนชราหรือคนพิการที่กำลังใช้ทาง ผู้ขับขี่ต้องใช้ความระมัดระวังเป็นพิเศษในการควบคุมรถของตน ดังนั้น หากผู้ขับขี่ขับรถชนคนเดินเท้า ผู้ขับขี่จะต้องรับผิดชอบต่อการกระทำดังกล่าว รวมถึงอาจต้องรับโทษตามกฎหมายที่เกี่ยวข้อง เช่น ประมวลกฎหมายอาญา หรือกฎหมายว่าด้วยการจราจรทางบก คำแนะนำ: ผู้ขับขี่ควรใช้ความระมัดระวังในการขับรถ โดยเฉพาะอย่างยิ่งเมื่อมีคนเดินเท้าอยู่บนถนน และต้องให้สัญญาณเตือนคนเดินเท้าให้รู้ตัวเมื่อจำเป็น เพื่อป้องกันการเกิดอุบัติเหตุและการรับผิดชอบต่อการกระทำของตนเอง


In [ ]:
response[: response.index('</s>')] if '</s>' in x else x

In [10]:
start = time.time()
response = legal_assistant.llm(llm_input)
end = time.time()
print(f'Execution Time: {end - start}')
print(response)

/lustrefs/disk/home/shanwibo/Capstone-TamTanai/src/llm/llm.py:61: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Execution Time: 41.5538489818573
ขับรถชนคนผิดไหม คำอธิบาย: มาตรา 31 กำหนดให้ผู้ขับขี่ต้องใช้ความระมัดระวังไม่ให้รถชนหรือโดนคนเดินเท้าไม่ว่าจะอยู่ ณ ส่วนใดของทาง และต้องให้สัญญาณเตือนคนเดินเท้าให้รู้ตัวเมื่อจำเป็น โดยเฉพาะอย่างยิ่ง เด็ก คนชราหรือคนพิการที่กำลังใช้ทาง ผู้ขับขี่ต้องใช้ความระมัดระวังเป็นพิเศษในการควบคุมรถของตน ดังนั้น หากผู้ขับขี่ขับรถชนคนเดินเท้า ผู้ขับขี่จะต้องรับผิดชอบต่อการกระทำดังกล่าว รวมถึงอาจต้องรับโทษตามกฎหมายที่เกี่ยวข้อง เช่น ประมวลกฎหมายอาญา หรือกฎหมายว่าด้วยการจราจรทางบก คำแนะนำ: ผู้ขับขี่ควรใช้ความระมัดระวังในการขับรถ โดยเฉพาะอย่างยิ่งเมื่อมีคนเดินเท้าอยู่บนถนน และต้องให้สัญญาณเตือนคนเดินเท้าให้รู้ตัวเมื่อจำเป็น เพื่อป้องกันการเกิดอุบัติเหตุและการรับผิดชอบต่อการกระทำของตนเอง</s>
 
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</s>
</


In [21]:
import pandas as pd
print(pd.read_csv('../asset/dataset/testdata.csv').iloc[0]['prompt'])

<s><|im_start|>system
คุณคือนักกฎหมายที่จะตอบคำถามเกี่ยวกับกฎหมาย จงตอบคำถามโดยใช้ความรู้ที่ให้ดังต่อไปนี้
ถ้าหากคุณไม่รู้คำตอบ ให้ตอบว่าไม่รู้ อย่าสร้างคำตอบขึ้นมาเอง
ความรู้ที่ให้:
พระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545

มาตรา 3  ในพระราชบัญญัตินี้
“ขายตรง” หมายความว่า การทำตลาดสินค้าหรือบริการในลักษณะของการนำเสนอขายต่อผู้บริโภคโดยตรง ณ ที่อยู่อาศัยหรือสถานที่ทำงานของผู้บริโภคหรือของผู้อื่น หรือสถานที่อื่นที่มิใช่สถานที่ประกอบการค้าเป็นปกติธุระ โดยผ่านตัวแทนขายตรงหรือผู้จำหน่ายอิสระชั้นเดียวหรือหลายชั้น แต่ไม่รวมถึงนิติกรรมตามที่กำหนดในกฎกระทรวง
“ตลาดแบบตรง”  หมายความว่า การทำตลาดสินค้าหรือบริการในลักษณะของการสื่อสารข้อมูลเพื่อเสนอขายสินค้าหรือบริการโดยตรงต่อผู้บริโภคซึ่งอยู่ห่างโดยระยะทางและมุ่งหวังให้ผู้บริโภคแต่ละรายตอบกลับเพื่อซื้อสินค้าหรือบริการจากผู้ประกอบธุรกิจตลาดแบบตรงนั้น ส่วนการซื้อขายสินค้าหรือบริการโดยวิธีการพาณิชย์อิเล็กทรอนิกส์ที่ไม่ถือว่าเป็นตลาดแบบตรง ให้เป็นไปตามหลักเกณฑ์และเงื่อนไขที่กำหนดในกฎกระทรวง
“ผู้บริโภค” หมายความว่า ผู้ซื้อหรือผู้ได้รับบริการจากผู้จำหน